[View in Colaboratory](https://colab.research.google.com/github/ZZANZU/DeeeeeepLearning/blob/master/cnn_mnist.ipynb)

In [1]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/datas

### GPU 작동 여부 체크

In [14]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [15]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 18082256047712434613, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11281989632
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 7145714452030838206
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [0]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

## CNN 계층 구성

In [0]:
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)

커널 크기를 2X2로 해서 2칸씩 움직임

In [0]:
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [0]:
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [0]:
W3 = tf.Variable(tf.random_normal([7 * 7 * 64, 256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 7 * 7 * 64])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)
L3 = tf.nn.dropout(L3, keep_prob)

In [0]:
W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L3, W4)

In [0]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

## 모델 학습

In [0]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [0]:
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

In [19]:
with tf.device('/gpu:0') :
    for epoch in range(15) :
        total_cost = 0

        for i in range(total_batch) :
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            batch_xs = batch_xs.reshape(-1, 28, 28, 1)

            _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs,
                                                                                            Y: batch_ys,
                                                                                            keep_prob: 0.7})
            total_cost += cost_val
        
        print('Epoch:', '%04d' % (epoch+1), 'Avg. cost =', '{:3f}'.format(total_cost / total_batch))
    
print('최적화 완료!')

Epoch: 0001 Avg. cost = 0.005823
Epoch: 0002 Avg. cost = 0.005892
Epoch: 0003 Avg. cost = 0.005204
Epoch: 0004 Avg. cost = 0.006500
Epoch: 0005 Avg. cost = 0.005401
Epoch: 0006 Avg. cost = 0.004369
Epoch: 0007 Avg. cost = 0.005022
Epoch: 0008 Avg. cost = 0.003652
Epoch: 0009 Avg. cost = 0.005517
Epoch: 0010 Avg. cost = 0.003845
Epoch: 0011 Avg. cost = 0.003563
Epoch: 0012 Avg. cost = 0.005224
Epoch: 0013 Avg. cost = 0.003983
Epoch: 0014 Avg. cost = 0.003489
Epoch: 0015 Avg. cost = 0.003521
최적화 완료!


In [12]:
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도 : ', sess.run(accuracy, feed_dict={X: mnist.test.images.reshape(-1, 28, 28, 1), 
                                                                        Y: mnist.test.labels,
                                                                        keep_prob: 1}))

정확도 :  0.9925
